# Training an Object Detector for Duckietown
By orlando.m.bol@gmail.com

YOLO is a fast object detector with decent accuracy. It is real-time and thus useful for the Duckietown environment. For more information, see https://pjreddie.com/darknet/yolo/.

## Clone a forked version of the YOLO3 repo that contains the Duckietown datasets

In [1]:
!git clone https://github.com/nasimnh/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 1602, done.
remote: Counting objects: 100% (1602/1602), done.
remote: Compressing objects: 100% (1220/1220), done.
remote: Total 1602 (delta 202), reused 1568 (delta 177), pack-reused 0
Receiving objects: 100% (1602/1602), 29.52 MiB | 23.67 MiB/s, done.
Resolving deltas: 100% (202/202), done.


## Install CUDA and compile Darknet (which YOLO is built on)

In [2]:
!wget -O cuda_9.2.88_396.26_linux.run -c https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda_9.2.88_396.26_linux

--2020-05-07 04:47:46--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda_9.2.88_396.26_linux
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda_9.2.88_396.26_linux.run?Z2rjmTODI_vh8u678-c1mbBo4iSLaTJxyP6xS8Q5qDbT2VwzfSxIvgB294r4zfKkcsQQEPU2nKhlNpfqeXgQQ_Tq_0fZHCXivAAZkwyqT0znHvAE_W_J-HFzjGjO7jXpvtIKZlzv2X6XN3VpXtFKQ5OW8DftJEWNDGCWpo1ygx7QEq-20h4QrdA [following]
--2020-05-07 04:47:47--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda_9.2.88_396.26_linux.run?Z2rjmTODI_vh8u678-c1mbBo4iSLaTJxyP6xS8Q5qDbT2VwzfSxIvgB294r4zfKkcsQQEPU2nKhlNpfqeXgQQ_Tq_0fZHCXivAAZkwyqT0znHvAE_W_J-HFzjGjO7jXpvtIKZlzv2X6XN3VpXtFKQ5OW8DftJEWNDGCWpo1ygx7QEq-20h4QrdA
Resolving developer.download.

In [3]:
!chmod +x cuda_9.2.88_396.26_linux.run
!./cuda_9.2.88_396.26_linux.run --verbose --silent --toolkit --override

Installing the CUDA Toolkit in /usr/local/cuda-9.2 ...
Verifying archive integrity... All good.

Uncompressing NVIDIA CUDA..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

**Include cuda in the PATH and compile Darknet**

Note that the Makefile has been modified to compile with CUDA and without OPENCV

In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda-9.2/bin'

In [5]:
cd darknet

/content/darknet


In [6]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -W

## Test YOLO installation

In [7]:
# Download pre-trained weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2020-05-07 04:53:36--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   134KB/s    in 15m 51s 

2020-05-07 05:09:29 (255 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [11]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

## Train tiny YOLO on Duckietown images
We realized that the more shallow version of YOLO, named tiny YOLO, is good enough for Duckietown. Since it has fewer convolutional layers, it trains and predicts faster. You can stop the training when you like and use the saved weights to do inference.

In [12]:
# Download pre-trained weights to effectively do transfer learning
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-05-07 05:34:51--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   210KB/s    in 12m 48s 

2020-05-07 05:47:42 (207 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [0]:
# Create a directory where we will save backups of the weights
!mkdir duckie_backup

In [19]:
# Train tiny YOLO using the already created configuration files
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detector train cfg/datafile.data cfg/yolov3-cup.cfg darknet53.conv.74

Streaming output truncated to the last 5000 lines.
Loaded: 0.000044 seconds
13821: 0.667916, 0.271957 avg, 0.001000 rate, 0.007463 seconds, 13821 images
Loaded: 0.012142 seconds
13822: 0.135907, 0.258352 avg, 0.001000 rate, 0.009321 seconds, 13822 images
Loaded: 0.011342 seconds
13823: 0.005800, 0.233097 avg, 0.001000 rate, 0.007506 seconds, 13823 images
Loaded: 0.010829 seconds
13824: 0.075950, 0.217382 avg, 0.001000 rate, 0.007785 seconds, 13824 images
Loaded: 0.010499 seconds
13825: 0.027608, 0.198405 avg, 0.001000 rate, 0.007454 seconds, 13825 images
Loaded: 0.012304 seconds
13826: 0.101196, 0.188684 avg, 0.001000 rate, 0.007515 seconds, 13826 images
Loaded: 0.011482 seconds
13827: 0.087250, 0.178541 avg, 0.001000 rate, 0.007533 seconds, 13827 images
Loaded: 0.010832 seconds
13828: 0.063773, 0.167064 avg, 0.001000 rate, 0.007704 seconds, 13828 images
Loaded: 0.010632 seconds
13829: 0.059009, 0.156258 avg, 0.001000 rate, 0.011646 seconds, 13829 images
Loaded: 0.009031 seconds
13830:

## Test your Duckietown Object Detector
Pick any image from the directory `data_folder/testset`

In [20]:
# Let's see what has been saved during training
!ls -hlt duckie_backup/

total 365M
-rw-r--r-- 1 root root 34M May  7 06:02 yolov3-cup.backup
-rw-r--r-- 1 root root 34M May  7 05:59 yolov3-cup_10000.weights
-rw-r--r-- 1 root root 34M May  7 05:56 yolov3-cup_900.weights
-rw-r--r-- 1 root root 34M May  7 05:56 yolov3-cup_800.weights
-rw-r--r-- 1 root root 34M May  7 05:55 yolov3-cup_700.weights
-rw-r--r-- 1 root root 34M May  7 05:55 yolov3-cup_600.weights
-rw-r--r-- 1 root root 34M May  7 05:55 yolov3-cup_500.weights
-rw-r--r-- 1 root root 34M May  7 05:55 yolov3-cup_400.weights
-rw-r--r-- 1 root root 34M May  7 05:55 yolov3-cup_300.weights
-rw-r--r-- 1 root root 34M May  7 05:55 yolov3-cup_200.weights
-rw-r--r-- 1 root root 34M May  7 05:55 yolov3-cup_100.weights


In [21]:
# Run inference
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detector test cfg/datafile.data cfg/yolov3-cup.cfg duckie_backup/yolov3-cup_10000.weights data_folder/testset/frame0044.jpg -thresh .50

layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max          2 x 2 / 1    13 x  13 x 512   ->    13 x  13 x 512
   12 con

## Transferring weights to Google Drive
You will want to transfer the trained weights to some other computer so that you can run inference. The easiest way is to copy them to your Google drive and then download the weights from there.

In [22]:
# Map your Google drive to Collab
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Copy weights to google drive
!cp duckie_backup/yolov3-cup.backup /content/gdrive/My\ Drive/yolov3-cup.backup